In [4]:
import sys
sys.path.append("./code")

In [5]:
from huggingface_hub import hf_hub_download
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

from quant import Quant

In [6]:
path_x = hf_hub_download(repo_id = f"monster-monash/WISDM", filename = f"WISDM_X.npy", repo_type = "dataset")
X = np.load(path_x, mmap_mode = "r")

path_y = hf_hub_download(repo_id = f"monster-monash/WISDM", filename = f"WISDM_y.npy", repo_type = "dataset")
y = np.load(path_y, mmap_mode = "r")

In [7]:
X_tr, X_te, Y_tr, Y_te = train_test_split(X, y, test_size=0.2, random_state=42)
X_tr = torch.tensor(X_tr.squeeze().astype(np.float32)).unsqueeze(1)
X_te = torch.tensor(X_te.squeeze().astype(np.float32)).unsqueeze(1)
le = LabelEncoder()
Y_tr = le.fit_transform(Y_tr)
Y_te = le.transform(Y_te)

quant = Quant(depth=6, div=4)
X_train_transform = quant.fit_transform(X_tr, Y_tr)

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3

In [ ]:
X_train_transform.shape

In [ ]:
classifier = ExtraTreesClassifier(n_estimators=200, max_features=0.1, criterion='entropy', n_jobs=-1)
classifier.fit(X_train_transform, Y_tr)

In [ ]:
X_test_transform = quant.transform(X_test)
predictions = classifier.predict(X_test_transform)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')